In [ ]:
# HuggingFace
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
# Prompts
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt,\
    PromptTemplate, FewShotPromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.schema import HumanMessage, SystemMessage,AIMessage
from langchain.prompts.example_selector import LengthBasedExampleSelector,\
    SemanticSimilarityExampleSelector
from langchain.chains.mapreduce import MapReduceChain
# Output parsing
from langchain.output_parsers import PydanticOutputParser,\
    OutputFixingParser,CommaSeparatedListOutputParser,\
        RetryWithErrorOutputParser,\
        StructuredOutputParser,ResponseSchema
# from pydantic import BaseModel,Field,field_validator
from langchain.pydantic_v1 import BaseModel, Field, validator
from typing import List,Dict
# Scapers,Retrievers,Loaders,Splitters,Embeddings
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.vectorstores import DeepLake
from langchain.chains import RetrievalQA,LLMChain,\
    ConversationalRetrievalChain, ConversationChain,\
        SimpleSequentialChain,RetrievalQAWithSourcesChain
from langchain.chains.base import Chain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter,\
    CharacterTextSplitter,NLTKTextSplitter,SpacyTextSplitter,\
    MarkdownTextSplitter,TokenTextSplitter,Language
from langchain.retrievers import ContextualCompressionRetriever
from langchain.document_loaders import PyPDFLoader, SeleniumURLLoader,\
    GoogleDriveLoader
from langchain.retrievers.document_compressors import LLMChainExtractor,\
    LLMChainFilter,EmbeddingsFilter
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain.docstore.document import Document
from langchain_cohere import CohereEmbeddings
# ConstitutionalAI
from langchain.chains.constitutional_ai.base import ConstitutionalChain
from langchain.chains.constitutional_ai.models import ConstitutionalPrinciple
# Conversation, Chat & Memory
from langchain.prompts.chat import ChatPromptTemplate,\
    SystemMessagePromptTemplate,\
        AIMessagePromptTemplate,HumanMessagePromptTemplate,\
        MessagesPlaceholder
from langchain import ConversationChain
from  langchain.memory import ConversationBufferMemory,\
    ConversationSummaryBufferMemory, ConversationSummaryMemory,\
    ConversationBufferWindowMemory
# Agents and Tools
from langchain.agents import tool # this is a decorator
from langchain import hub
from langchain.agents import AgentType,initialize_agent,\
    load_tools, Tool,\
        AgentExecutor,\
        create_json_agent,create_react_agent,create_structured_chat_agent
from langchain_google_community import GoogleSearchAPIWrapper
from langchain.tools import DuckDuckGoSearchRun,DuckDuckGoSearchResults
from langchain_community.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain_experimental.utilities import PythonREPL
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain_experimental.autonomous_agents import BabyAGI, AutoGPT
from langchain.tools.file_management import WriteFileTool,ReadFileTool
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
import faiss
# Others
from newspaper import Article, Config
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup
from elevenlabs.client import ElevenLabs
from streamlit_chat import message
from audio_recorder_streamlit import audio_recorder
import streamlit as st
import cohere
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import seaborn as sns
import json
import requests
import os
import json
import yaml
import yt_dlp
import whisper
import textwrap
import re
import tiktoken
import time
# import spacy
# spacy.load('en_core_web_sm')
# warnings.filterwarnings("ignore")

In [2]:
from dotenv import load_dotenv

# Provide the filename as a string
load_dotenv('../.env')

True

In [ ]:
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

repo_id = "mistralai/Mistral-7B-Instruct-v0.3"

model_kwargs = {
    "max_new_tokens": 1000, # Maximum tokens to generate
    "max_length": 3000, # Maximum length of input + output
    "temperature": 0.1, # Controls randomness of output
    "timeout": 6000,
    # "task":'conversational',
}


llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    huggingfacehub_api_token = HUGGINGFACEHUB_API_TOKEN,
    # you specify the task or not
    # You can also specify the task in the model_kwargs or within here
    # task = 'conversational',
    **model_kwargs,
)

In [4]:
desktop = os.path.expanduser("~\Desktop")
llmai = os.path.join(desktop,"llmai")
llm = os.path.join(llmai,"llms")
projects = os.path.join(llm,"my_llm_projects")
resources = os.path.join(projects,"resources")
pdf_book = os.path.join(resources,"deepak-chopra-the-7-laws-of-success.pdf")

if  os.path.exists(pdf_book):
    print("File path is okay!")
else:
    raise ValueError(f"The file at path {pdf_book} does not exist.")

File path is okay!


### **Character Text Splitter**

PyPDFLoader and CharacterTextSplitter are somewhat the same

In [32]:
loader = PyPDFLoader(pdf_book)
pages = loader.load_and_split()

In [44]:
# print(pages)
print(len(pages))
print(pages[3].dict().keys())
print(pages[3].page_content)
print(pages[3].metadata.keys())
# print(pages[3].metadata['source'])
print(pages[3].metadata['page'])
print(pages[3].type)

43
dict_keys(['page_content', 'metadata', 'type'])
INTRODUCTION
Although this book is titled The Seven Spiritual Laws of Success,  it could also be called The
Seven Spiritual Laws of Life,  because these are the same principles that nature uses to create
everything in material existence  everything we can see, hear, smell, taste, or touch.
In my book, Creating Affluence: Wealth Consciousness in the Field of All Possibilities,  I
have outlined the steps to wealth consciousness based on a true understanding of the workingsof nature. The Seven Spiritual Laws of  Success form the essence of this teaching. When this
knowledge is incorporated in your consciousness, it will give you the ability to create unlimited
wealth with effortless ease, and to experience success in every endeavour.
Success in life could be defined as the continued expansion of happiness and the progres-
sive realization of worthy goals. Success is the ability to fulfill your desires with effortless ease.And yet success

In [34]:
text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
texts = text_splitter.split_documents(pages)

In [45]:
# print(texts)
print(len(texts))
print(texts[3].dict().keys())
print(texts[3].page_content)
print(texts[3].metadata.keys())
# print(texts[3].metadata['source'])
print(texts[3].metadata['page'])
print(texts[3].type)

43
dict_keys(['page_content', 'metadata', 'type'])
INTRODUCTION
Although this book is titled The Seven Spiritual Laws of Success,  it could also be called The
Seven Spiritual Laws of Life,  because these are the same principles that nature uses to create
everything in material existence  everything we can see, hear, smell, taste, or touch.
In my book, Creating Affluence: Wealth Consciousness in the Field of All Possibilities,  I
have outlined the steps to wealth consciousness based on a true understanding of the workingsof nature. The Seven Spiritual Laws of  Success form the essence of this teaching. When this
knowledge is incorporated in your consciousness, it will give you the ability to create unlimited
wealth with effortless ease, and to experience success in every endeavour.
Success in life could be defined as the continued expansion of happiness and the progres-
sive realization of worthy goals. Success is the ability to fulfill your desires with effortless ease.And yet success

### **Recursive Character Text Splitter**

In [36]:
recursive_text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=100)
recursive_texts = recursive_text_splitter.split_documents(pages)

In [46]:
# print(recursive_texts)
print(len(recursive_texts))
print(recursive_texts[3].dict().keys())
print(recursive_texts[3].page_content)
print(recursive_texts[3].metadata.keys())
# print(recursive_texts[3].metadata['source'])
print(recursive_texts[3].metadata['page'])
print(recursive_texts[3].type)

132
dict_keys(['page_content', 'metadata', 'type'])
ACKNOWLEDGMENTS
I would like to express my love and gratitude to the following people:
Janet Mills for lovingly nurturing this book from conception to completion.
Rita Chopra, Mallika Chopra, and Gautama Chopra for being the living expression of The
Seven Spiritual Laws.
Ray Chambers, Gayle Rose, Adrianna Nienow, David Simon, George Harrison, Olivia
Harrison, Naomi Judd, Demi Moore, and Alice Walton for their courage and commitment to a
vision that is awesome, inspiring, lofty, noble, and life-transforming.
Roger Gabriel, Brent Becvar, Rose Bueno-Mur-phy, and all my staff at the Sharp Center
for MindBody Medicine for being inspiring examples to all our guests and patients.
Deepak Singh, Geeta Singh, and all my staff at Quantum Publications for their unflagging
energy and dedication.
Muriel Nellis for her unflinching intention to maintain the highest level of integrity in all our
endeavors.
dict_keys(['source', 'page'])
2
Document


In [47]:
print(recursive_texts[4].page_content)

endeavors.
Richard Peri for being such a great example of self-referral.Arielle Ford for her unshakable faith in self-knowledge and her infectious enthusiasm and
commitment to transform the lives of so many people.
And Bill Elkus for his understanding and friendship.


### **NLTK Text Splitter**

In [40]:
desktop = os.path.expanduser("~\Desktop")
llmai = os.path.join(desktop,"llmai")
llm = os.path.join(llmai,"llms")
projects = os.path.join(llm,"my_llm_projects")
resources = os.path.join(projects,"resources")
text_source = os.path.join(resources,"my_medium_articles.txt")

if  os.path.exists(text_source):
    print("File path is okay!")
else:
    raise ValueError(f"The file at path {text_source} does not exist.")

File path is okay!


In [57]:
nltk_text_splitter = NLTKTextSplitter(chunk_size=1000, chunk_overlap=100)
with open(text_source) as my_file_reader:
    my_file = my_file_reader.read()
nltk_texts = nltk_text_splitter.split_text(my_file)

In [64]:
try:
    print(nltk_texts)
    print(nltk_texts[3])
    print(len(nltk_texts))
except IndexError as error:
    print(f"{len(nltk_texts)}, please change the chunk_size or change your index")

['Title:Recurrent Neural Network, LSTM and Bidirectional LSTM\nArticle:\nPhoto by Gülfer ERGİN on Unsplash\n\nRecurrent Neural Network, LSTM and Bidirectional LSTM Daniel Chiebuka Ihenacho · Follow 4 min read · Jan 5, 2023 -- Listen Share\n\nRecent innovations in Natural Language Processing have focused on how the context of words can be properly captured in their vector representations.\n\nTraditional neural networks cannot do this with high accuracy and for this reason Recurrent neural networks have been developed to address this issue due to the following;\n\nUse of sequential information.\n\nPresence of memory\n\nThe above diagram is a Rolled RNN, where;\n\nA is the Rolled neural network\n\nis the Rolled neural network xₜ is a given input\n\nis a given input hₜ is the output.\n\nis the output.\n\nThe loop allows information to be passed onto the next step of the network.', 'is the output.\n\nThe loop allows information to be passed onto the next step of the network.\n\nThe diagram 

### **SpacyTextSplitter**

In [67]:
import spacy
spacy.load('en_core_web_sm')

In [88]:
spacy_text_splitter = SpacyTextSplitter(chunk_size=1000, 
                                        chunk_overlap=100)
                                        # separator="") # less texts: not good
                                        # separator="".join(['\n','\n']))
with open(text_source) as my_file_reader:
    my_file = my_file_reader.read()
spacy_texts = spacy_text_splitter.split_text(my_file)

In [89]:
try:
    print(spacy_texts)
    print(spacy_texts[3])
    print(len(spacy_texts))
except IndexError as error:
    print(f"{len(spacy_texts)}, please change the chunk_size or change your index")

['Title:Recurrent Neural Network, LSTM and Bidirectional LSTM\nArticle:\nPhoto by Gülfer ERGİN on Unsplash\n\nRecurrent Neural Network, LSTM and Bidirectional LSTM Daniel Chiebuka Ihenacho · Follow 4 min read · Jan 5, 2023 -- Listen Share\n\nRecent innovations in Natural Language Processing have focused on how the context of words can be properly captured in their vector representations.\n\nTraditional neural networks cannot do this with high accuracy and for this reason Recurrent neural networks have been developed to address this issue due to the following;\n\nUse of sequential information.\n\n\n\nPresence of memory\n\nThe above diagram is a Rolled RNN, where;\n\nA is the Rolled neural network\n\nis the Rolled neural network xₜ is a given input\n\nis a given input hₜ is the output.\n\n\n\nis the output.\n\nThe loop allows information to be passed onto the next step of the network.', 'is the output.\n\nThe loop allows information to be passed onto the next step of the network.\n\n\n\n

### **MarkdownTextSplitter**

In [98]:
markdown_text = """
# 

# Welcome to My Blog!

## Introduction
Hello everyone! My name is **John Doe** and I am a _software developer_. I specialize in Python, Java, and JavaScript.

Here's a list of my favorite programming languages:

1. Python
2. JavaScript
3. Java

You can check out some of my projects on [GitHub](https://github.com).

## About this Blog
In this blog, I will share my journey as a software developer. I'll post tutorials, my thoughts on the latest technology trends, and occasional book reviews.

Here's a small piece of Python code to say hello:

\``` python
def say_hello(name):
    print(f"Hello, {name}!")

say_hello("John")
\```

Stay tuned for more updates!

## Contact Me
Feel free to reach out to me on [Twitter](https://twitter.com) or send me an email at johndoe@email.com.

"""

markdown_splitter = MarkdownTextSplitter(chunk_size=100,chunk_overlap=0)
markdown_docs = markdown_splitter.create_documents([markdown_text])


In [109]:
# print(markdown_docs)
print(len(markdown_docs))
print(markdown_docs[2].dict().keys())
print(markdown_docs[2].page_content)
print(markdown_docs[2].metadata.keys())
# print(markdown_docs[2].metadata['source'])
print(markdown_docs[2].type)

15
dict_keys(['page_content', 'metadata', 'type'])
Hello everyone! My name is **John Doe** and I am a _software developer_. I specialize in Python,
dict_keys([])
Document


### **TokenTextSplitter**

One potential drawback of using `TokenTextSplitter` is that it may require additional computation when converting text to BPE tokens and back. If you need a faster and simpler text-splitting method, you might consider using `CharacterTextSplitter`, which directly splits the text based on character count, offering a more straightforward approach to text segmentation.

In [107]:
token_text_splitter = TokenTextSplitter(chunk_size=1000,chunk_overlap=100)
token_texts = text_splitter.split_text(my_file)

Created a chunk of size 1313, which is longer than the specified 1000
Created a chunk of size 1254, which is longer than the specified 1000
Created a chunk of size 1185, which is longer than the specified 1000


In [108]:
try:
    print(token_texts)
    print(token_texts[3])
    print(len(token_texts))
except IndexError as error:
    print(f"{len(token_texts)}, please change the chunk_size or change your index")

['Title:Recurrent Neural Network, LSTM and Bidirectional LSTM\nArticle:\nPhoto by GÃ¼lfer ERGÄ°N on Unsplash\n\nRecurrent Neural Network, LSTM and Bidirectional LSTM Daniel Chiebuka Ihenacho Â· Follow 4 min read Â· Jan 5, 2023 -- Listen Share\n\nRecent innovations in Natural Language Processing have focused on how the context of words can be properly captured in their vector representations. Traditional neural networks cannot do this with high accuracy and for this reason Recurrent neural networks have been developed to address this issue due to the following;\n\nUse of sequential information.\n\nPresence of memory\n\nThe above diagram is a Rolled RNN, where;\n\nA is the Rolled neural network\n\nis the Rolled neural network xâ\x82\x9c is a given input\n\nis a given input hâ\x82\x9c is the output.\n\nis the output. The loop allows information to be passed onto the next step of the network.\n\nThe diagram shows the unrolled RNN network', 'The diagram shows the unrolled RNN network\n\nThe

### **Splitting code**

In [120]:
# Full list of supported languages
[e.value for e in Language]

['cpp',
 'go',
 'java',
 'kotlin',
 'js',
 'ts',
 'php',
 'proto',
 'python',
 'rst',
 'ruby',
 'rust',
 'scala',
 'swift',
 'markdown',
 'latex',
 'html',
 'sol',
 'csharp',
 'cobol',
 'c',
 'lua',
 'perl',
 'haskell',
 'elixir']

In [113]:
JS_CODE = """
// Define a function to print a greeting message
function helloWorld() {
  console.log("Hello, World!");
}

// Define a function to print a personalized greeting
function greet(name) {
  console.log("Hello, " + name + "!");
}

// Call the helloWorld function
helloWorld();

// Call the greet function with different names
greet("Alice");
greet("Bob");
greet("Charlie");

// A function to demonstrate a simple arithmetic operation
function addNumbers(a, b) {
  console.log("The sum of " + a + " and " + b + " is " + (a + b) + ".");
}

// Call the addNumbers function
addNumbers(5, 7);
addNumbers(10, 20);

"""

js_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.JS, chunk_size=60, chunk_overlap=0
)
js_docs = js_splitter.create_documents([JS_CODE])

In [115]:
# print(js_docs)
print(len(js_docs))
print(js_docs[2].dict().keys())
print(js_docs[2].page_content)
print(js_docs[2].metadata.keys())
# print(js_docs[2].metadata['source'])
print(js_docs[2].type)

15
dict_keys(['page_content', 'metadata', 'type'])
// Define a function to print a personalized greeting
dict_keys([])
Document


In [116]:
PYTHON_CODE = """
# Define a function to print a greeting message
def hello_world():
    print("Hello, World!")

# Define a function to print a personalized greeting
def greet(name):
    print(f"Hello, {name}!")

# Call the hello_world function
hello_world()

# Call the greet function with different names
greet("Alice")
greet("Bob")
greet("Charlie")

# A function to demonstrate a simple arithmetic operation
def add_numbers(a, b):
    print(f"The sum of {a} and {b} is {a + b}.")

# Call the add_numbers function
add_numbers(5, 7)
add_numbers(10, 20)


"""

py_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=60, chunk_overlap=0
)
py_docs = py_splitter.create_documents([PYTHON_CODE])

In [119]:
# print(py_docs)
print(len(py_docs))
print(py_docs[2].dict().keys())
print(py_docs[2].page_content)
print(py_docs[2].metadata.keys())
# print(py_docs[2].metadata['source'])
print(py_docs[2].type)

12
dict_keys(['page_content', 'metadata', 'type'])
# Define a function to print a personalized greeting
dict_keys([])
Document
